# 🎬 Age & Emotion Detection for Horror Movie Theatre (Rule-Based Gate)

**Goal**: Real-time system that detects **age** for each face. If **age < 13** or **age > 60** → show **"Not allowed"** and draw **red** box. If **13 ≤ age ≤ 60** → also predict **emotion**.

All predictions (age, emotion if applicable, entry time) are **logged to CSV**.

👉 This notebook **creates its own ML model**: a compact CNN trained quickly on a **synthetic face-like dataset** (so it runs anywhere without downloading large datasets). You can later swap in a real dataset (e.g., UTKFace for age + FER2013 for emotion) using the data loaders provided.

### What you get
- **Custom PyTorch model** with a shared backbone and two heads: **age regression** + **emotion classification** (4 classes: `neutral, happy, sad, scared`).
- Fast training on a small **synthetic dataset** (generated on the fly).
- **OpenCV**-based face detection (Haar cascade) for real-time/video/image inference.
- **CSV logging**: `logs/theatre_entries.csv` with columns `[timestamp, person_id, age_pred, emotion_pred, allowed]`.

---
## Quick Start
1. Run the **Install** cell.
2. Run **Create Synthetic Dataset** (or plug in your own data path).
3. Run **Train** (few epochs).
4. Run **Live/Video/Image Inference**.

**Note**: For headless environments (e.g., Colab), set `SHOW_WINDOWS=False` to avoid GUI errors; frames will be saved to `out/`.


In [ ]:
!pip install torch torchvision torchaudio --quiet
!pip install opencv-python pandas scikit-learn --quiet

In [ ]:
import os, math, time, csv, random, shutil
from datetime import datetime
import numpy as np
import pandas as pd
import cv2
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from sklearn.model_selection import train_test_split

DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
SEED = 42
random.seed(SEED); np.random.seed(SEED); torch.manual_seed(SEED)

os.makedirs('data/synth/train', exist_ok=True)
os.makedirs('data/synth/val', exist_ok=True)
os.makedirs('models', exist_ok=True)
os.makedirs('logs', exist_ok=True)
os.makedirs('out', exist_ok=True)

SHOW_WINDOWS = False  # set True if running locally with display
HAAR_FACE = cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
FACE_CASCADE = cv2.CascadeClassifier(HAAR_FACE)
EMOTION_LABELS = ['neutral','happy','sad','scared']
IMG_SIZE = 64


## Synthetic dataset generator (runs anywhere)
We procedurally draw simple face-like emojis with parameters that correlate with **age** (wrinkles/skin texture) and **emotion** (mouth/eyebrow shapes).

In [ ]:
def draw_face(age: int, emotion: str, size=IMG_SIZE):
    img = np.zeros((size, size, 3), dtype=np.uint8) + 255
    # base face ellipse
    cv2.ellipse(img, (size//2, size//2), (size//3, size//3), 0, 0, 360, (220, 200, 180), -1)
    # eyes
    eye_y = size//2 - size//6
    eye_x_off = size//6
    cv2.circle(img, (size//2 - eye_x_off, eye_y), size//20, (0,0,0), -1)
    cv2.circle(img, (size//2 + eye_x_off, eye_y), size//20, (0,0,0), -1)
    # mouth based on emotion
    mx, my = size//2, size//2 + size//8
    w = size//4
    if emotion == 'happy':
        cv2.ellipse(img, (mx,my), (w, w//3), 0, 0, 180, (0,0,0), 2)
    elif emotion == 'sad':
        cv2.ellipse(img, (mx,my+4), (w, w//3), 0, 180, 360, (0,0,0), 2)
    elif emotion == 'scared':
        cv2.circle(img, (mx,my), w//4, (0,0,0), 2)
    else:
        cv2.line(img, (mx-w, my), (mx+w, my), (0,0,0), 2)
    # age cues: wrinkles for older, smooth for young
    if age > 50:
        for k in range(3):
            y = size//2 + k*4
            cv2.line(img, (mx-w//2, y), (mx+w//2, y), (160,140,120), 1)
    if age < 16:
        cv2.circle(img, (size//2 - eye_x_off, eye_y+10), 3, (255,180,180), -1)
        cv2.circle(img, (size//2 + eye_x_off, eye_y+10), 3, (255,180,180), -1)
    # slight noise to diversify
    noise = np.random.normal(0, 3, img.shape).astype(np.int16)
    img = np.clip(img.astype(np.int16) + noise, 0, 255).astype(np.uint8)
    return img

class SynthFaceDataset(Dataset):
    def __init__(self, n=2000, transform=None):
        self.transform = transform
        self.ages = np.random.randint(3, 85, size=n)
        self.emotions = np.random.choice(EMOTION_LABELS, size=n)
    def __len__(self):
        return len(self.ages)
    def __getitem__(self, idx):
        age = int(self.ages[idx])
        emo = str(self.emotions[idx])
        img = draw_face(age, emo)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        x = torch.tensor(img).permute(2,0,1).float()/255.0
        age_norm = torch.tensor([age/100.0], dtype=torch.float32)
        emo_idx = torch.tensor(EMOTION_LABELS.index(emo), dtype=torch.long)
        return x, age_norm, emo_idx


## Model: Tiny CNN with shared backbone + 2 heads

In [ ]:
import torch.nn as nn

class TinyBackbone(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(3, 16, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(16, 32, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(), nn.MaxPool2d(2),
            nn.Conv2d(64, 128, 3, padding=1), nn.ReLU(), nn.AdaptiveAvgPool2d((1,1))
        )
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return x

class AgeEmotionNet(nn.Module):
    def __init__(self, num_emotions=4):
        super().__init__()
        self.backbone = TinyBackbone()
        self.age_head = nn.Sequential(nn.Linear(128, 64), nn.ReLU(), nn.Linear(64,1))
        self.emo_head = nn.Sequential(nn.Linear(128, 64), nn.ReLU(), nn.Linear(64,num_emotions))
    def forward(self, x):
        feat = self.backbone(x)
        age = self.age_head(feat)
        emo = self.emo_head(feat)
        return age, emo

model = AgeEmotionNet(4)
sum(p.numel() for p in model.parameters())


## Training

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'
BATCH = 64; LR = 1e-3; EPOCHS = 5
train_ds = SynthFaceDataset(2000)
val_ds = SynthFaceDataset(400)
train_dl = DataLoader(train_ds, batch_size=BATCH, shuffle=True)
val_dl = DataLoader(val_ds, batch_size=BATCH)
model = AgeEmotionNet(4).to(DEVICE)
opt = torch.optim.Adam(model.parameters(), lr=LR)
mse, ce = nn.MSELoss(), nn.CrossEntropyLoss()

def one_epoch(dl, train=True):
    model.train(train)
    tot, n = 0.0, 0
    for x, age, emo in dl:
        x, age, emo = x.to(DEVICE), age.to(DEVICE), emo.to(DEVICE)
        if train: opt.zero_grad()
        age_p, emo_p = model(x)
        loss = mse(age_p, age) + ce(emo_p, emo)
        if train: loss.backward(); opt.step()
        tot += loss.item()*x.size(0); n += x.size(0)
    return tot/max(n,1)

for e in range(1, EPOCHS+1):
    tr = one_epoch(train_dl, True)
    va = one_epoch(val_dl, False)
    print(f"Epoch {e}/{EPOCHS}  train:{tr:.4f}  val:{va:.4f}")
torch.save(model.state_dict(), 'models/age_emo_synth.pt')
print('Saved to models/age_emo_synth.pt')


## Inference & CSV logging

In [ ]:
CSV_LOG = 'logs/theatre_entries.csv'
FACE_CASCADE = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

def load_model(path='models/age_emo_synth.pt'):
    m = AgeEmotionNet(4).to(DEVICE)
    m.load_state_dict(torch.load(path, map_location=DEVICE))
    m.eval(); return m

def predict_one_face(face_bgr, m):
    face_rgb = cv2.cvtColor(cv2.resize(face_bgr, (64,64)), cv2.COLOR_BGR2RGB)
    x = torch.tensor(face_rgb).permute(2,0,1).float()/255.0
    with torch.no_grad():
        age_p, emo_p = m(x.unsqueeze(0).to(DEVICE))
        age = float(np.clip(age_p.item()*100.0, 0, 100))
        emo = ['neutral','happy','sad','scared'][int(torch.argmax(emo_p,1))]
    return age, emo

def log_entry(person_id, age, emotion_or_na, allowed):
    t = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    exists = os.path.exists(CSV_LOG)
    with open(CSV_LOG, 'a', newline='') as f:
        w = csv.writer(f)
        if not exists:
            w.writerow(['timestamp','person_id','age_pred','emotion_pred','allowed'])
        w.writerow([t, person_id, f"{age:.1f}", emotion_or_na, int(allowed)])

def annotate_frame(frame, m, next_id_start=1):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = FACE_CASCADE.detectMultiScale(gray, 1.2, 5)
    pid = next_id_start
    for (x,y,w,h) in faces:
        face = frame[y:y+h, x:x+w]
        age, emo = predict_one_face(face, m)
        allowed = (13 <= age <= 60)
        color = (0,255,0) if allowed else (0,0,255)
        label = f"{age:.0f} - {emo if allowed else 'Not allowed'}"
        cv2.rectangle(frame, (x,y), (x+w, y+h), color, 2)
        cv2.putText(frame, label, (x, max(0,y-8)), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        log_entry(pid, age, emo if allowed else 'N/A', allowed)
        pid += 1
    return frame

model = load_model()

# Example usage:
# img = cv2.imread('some_image.jpg'); out = annotate_frame(img, model); cv2.imwrite('out/result.jpg', out)
# For video/webcam, use the helper below:
def run_source(src=0, show_windows=False):
    cap = cv2.VideoCapture(src)
    i=0
    while True:
        ret, frame = cap.read();
        if not ret: break
        out = annotate_frame(frame, model, 1)
        if show_windows:
            cv2.imshow('Theatre Gate', out)
            if cv2.waitKey(1) & 0xFF == 27: break
        else:
            if i % 30 == 0:
                cv2.imwrite(f'out/frame_{i:05d}.jpg', out)
        i += 1
    cap.release();
    if show_windows: cv2.destroyAllWindows()

# run_source(0, show_windows=False)
